In [1]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
import math
import pickle

Welcome to JupyROOT 6.15/01


In [2]:
ROOT.gROOT.ProcessLine("#include <math.h>");
ROOT.gSystem.Load("libDelphes");

In [3]:
from CutAnalyzer_lib import save_obj

In [4]:
%jsroot on

In [5]:
#If input samples is a list use 0 if is just one file use 1 on TSam variable
def ChainConstructor(path, ListOfFolders):
    DTrees={};
    for Folder in ListOfFolders:
        #Path=path+Folder+"/*_events1.root"
        Path=path+Folder+"/*.root"
        MainChain=ROOT.TChain("Delphes");
        MainChain.Add(Path)
        print "add ", Path
        DTrees[Folder]=MainChain;
    return DTrees

In [6]:
#The backgrounds
#Path1= "/home/rsiii/SSSFDM/VLF/Background/"; 
Path1= "/home/camilo/cms/Scripts_MadGraph_VLFTEO/Datos/BackGround-llW/Events/"
ListOfFolders = ["run_01","run_02"];
#The Signal
Path2= "/home/camilo/cms/Scripts_MadGraph_VLFTEO/Datos/BackGround-tW/Events"; 
FolderSignal=["run_01","run_02"];

In [8]:
DBack

{'run_01': <ROOT.TChain object ("Delphes") at 0x55761a3cf1b0>,
 'run_02': <ROOT.TChain object ("Delphes") at 0x557617fee140>}

In [9]:
DBack={}
DTrees={}
DSignal={}
DBack=ChainConstructor(Path1,ListOfFolders)
DSignal=ChainConstructor(Path2,FolderSignal)
DTrees["BckgndW+Jets"]=DBack["run_01"]
#DTrees["BckgndSingleTop"]=DBack["singletop"]
DTrees["Signal"]=DSignal["run_01"]

add  /home/camilo/cms/Scripts_MadGraph_VLFTEO/Datos/BackGround-llW/Events/run_01/*.root
add  /home/camilo/cms/Scripts_MadGraph_VLFTEO/Datos/BackGround-llW/Events/run_02/*.root
add  /home/camilo/cms/Scripts_MadGraph_VLFTEO/Datos/BackGround-tW/Eventsrun_01/*.root
add  /home/camilo/cms/Scripts_MadGraph_VLFTEO/Datos/BackGround-tW/Eventsrun_02/*.root


In [10]:
LsOfCuts={
            "Jet":
                {
                    #Leading Jet PT greater than 60GeV, and all of them less than 100
                    "PT":[(0,">60"), "<100"],
                    "Entries":[">1","<3"]
                },
            "Muon":
                {
                    #Leading Muon PT greater than 5GeV
                    "PT":[(0,">5")],
                    #number of Muon equal to one
                    "Entries":["==1"]
                },
            "MissingET":
                {
                    #
                    "MET":[">60"]
                }
        }

In [ ]:
LsOfCuts={
            "Jet":
                {
                    #Leading Jet PT greater than 60GeV, and all of them less than 100
                    "PT":[(0,">60")]
                },
            "Muon":
                {
                    #Leading Muon PT greater than 5GeV
                    "PT":[(0,">5")],
                    #number of Muon equal to one
                    "Entries":["==1"]
                },
            "MissingET":
                {
                    #
                    "MET":[">60"]
                }
        }

In [11]:
LsOfCuts

{'Jet': {'Entries': ['>1', '<3'], 'PT': [(0, '>60'), '<100']},
 'MissingET': {'MET': ['>60']},
 'Muon': {'Entries': ['==1'], 'PT': [(0, '>5')]}}

In [12]:
Chain=DTrees["Signal"]

In [13]:
LsOfPOandVOI={
                "Jet":[(0,"PT"),(1,"PT"),"Entries"],
                "Muon":[(0,"PT"),"Entries"],
                "MissingET":["MET"]
                }

In [14]:
LsOfPOandVOI

{'Jet': [(0, 'PT'), (1, 'PT'), 'Entries'],
 'MissingET': ['MET'],
 'Muon': [(0, 'PT'), 'Entries']}

In [19]:
Debug=False
Debug=True

In [20]:
    #CutFlow dictionary, to keep the track of the cuts the events pass or fail 
    CutFlow={}
    #Set the deafault cutflow 
    CutFlowUser=[]
    PhOb=LsOfCuts.keys()
    #print "PhOb", PhOb
    #If is empty return error
    for i in PhOb:
        VaOfIn=LsOfCuts[i]
        #If is empty return error
        #print VaOfIn
        Cuts=VaOfIn.keys()
        #If is empty return error
        #print "Cuts", Cuts
        for j in Cuts:
            cut_base=VaOfIn[j]
            for cut_ in cut_base:
                if isinstance(cut_, tuple):
                    #print i+j+cut_[1]
                    CutFlowUser.append("Count_"+i+"["+str(cut_[0])+"]"+j+cut_[1])
                else:
                    #print i+j+cut_
                    if i=="Jet" or i=="Muon" or i=="Electron" or i=="Photon":
                        if j != "Entries":
                            CutFlowUser.append("Count_"+i+"[0]"+j+cut_)
                        else:
                            CutFlowUser.append("Count_"+i+j+cut_)
                    else:
                        CutFlowUser.append("Count_"+i+j+cut_)
    #If the cutflow have this number of Trues entries 
    Validationlen=len(CutFlowUser)
    
    
    #Create the plots
    Histograms={}
    CutUserFlagsObj= LsOfPOandVOI.keys()
    
    #The ExRootTreeReader objet
    treeReader = ROOT.ExRootTreeReader(Chain)
    numberOfEntries = treeReader.GetEntries()
        
   
    counts={}
    #PhOb=LsOfCuts.keys()
    PhyObj={}
    #Branches=["Jet","Muon","MissingET","ScalarHT","Photon","Electron"]
    #get the branches I am going to use
    for branch in PhOb:
        PhyObj[branch] = treeReader.UseBranch(branch)
        
    # Loop over events
    for entry in range(0, numberOfEntries):
        if Debug: 
            if entry==100:
                break
        flags={}
        #Get the entry
        treeReader.ReadEntry(entry)
        
        # Cuts over jets
        if LsOfCuts.has_key('Jet'):                         #Cuts over Jets?
            if Debug:
                print "Jet"
            if PhyObj["Jet"].GetEntries() > 0:              #There are any Jets in the data
                VaOfIn=LsOfCuts['Jet']                      #Which cuts over jets
                
                if VaOfIn.has_key('Entries'):
                    if Debug:
                        print "Entries"
                    Cuts=VaOfIn['Entries']
                    for cut_ in Cuts:               #run over all the Entries cuts 
                        flag=False
                        filt=cut_
                        #Evaluate the condition
                        mycode="flag = True if PhyObj['Jet'].GetEntries() "+filt+" else False" 
                        if Debug:
                            print mycode
                        exec(mycode)
                        name="Count_JetEntries"+filt
                        if flag:
                            if Debug:
                                print "Pass: "+filt
                            #Counts
                            if counts.has_key(name):
                                counts[name] += 1
                            else:
                                counts[name] = 1
                        #CutFlow    
                            flags[name] = True  
                        else:
                            if Debug:
                                print "Fail: "+filt
                            flags[name] = False 
                                            
                if VaOfIn.has_key('PT') and PhyObj["Jet"].GetEntries() > 0:                    #If ther are cut over PT
                    if Debug:
                        print "PT"
                    Cuts=VaOfIn['PT']
                    for cut_ in Cuts:               #run over all the PT cuts 
                        if isinstance(cut_, tuple):     #See if is tuple
                            if Debug:
                                print "Tuple"
                            flag=False
                            filt=cut_[1]
                            #See if there are enough entries to evaluate
                            if(PhyObj["Jet"].GetEntries() > cut_[0]):
                                jet= PhyObj["Jet"].At(cut_[0])
                                #Evaluate the condition
                                mycode="flag = True if jet.PT "+filt+" else False" 
                                exec(mycode)
                                name="Count_Jet["+str(cut_[0])+"]PT"+cut_[1]   
                            if flag:                    #Counts
                                if counts.has_key(name):
                                    counts[name] += 1
                                else:
                                    counts[name] = 1
                                #CutFlow    
                                flags[name] = True  
                            else:
                                flags[name] = False 
                            
                        else:           #Evaluate the condition for the avaible jets 
                            for l in range(0, PhyObj["Jet"].GetEntries() ):
                                jet = PhyObj["Jet"].At(l)
                                flag=False
                                filt=cut_
                                #Evaluate the condition
                                mycode="flag = True if jet.PT "+filt+" else False" 
                                exec(mycode)
                                name="Count_Jet["+str(l)+"]PT"+filt
                                if flag:                    #Counts
                                    if counts.has_key(name):
                                        counts[name] += 1
                                    else:
                                        counts[name] = 1
                                #CutFlow    
                                    flags[name] = True  
                                else:
                                    flags[name] = False
                
                if VaOfIn.has_key('Eta'):
                    print "Jet Eta Not implementet yet"
                if VaOfIn.has_key('Phi'):
                    print "Jet Phi Not implementet yet"
                if VaOfIn.has_key('Mass'):
                    print "Jet Mass Not implementet yet"
                if VaOfIn.has_key('Flavor'):
                    print "Jet Flavor Not implementet yet"
                if VaOfIn.has_key('FlavorAlgo'):
                    print "Jet FlavorAlgo Not implementet yet"
                if VaOfIn.has_key('FlavorPhys'):
                    print "Jet FlavorPhys Not implementet yet"
                if VaOfIn.has_key('BTag'):
                    print "Not BTag implementet yet"
                if VaOfIn.has_key('BTagAlgo'):
                    print "Jet BTagAlgo Not implementet yet"
                if VaOfIn.has_key('BTagPhys'):
                    print "Jet BTagPhys Not implementet yet"
                if VaOfIn.has_key('TauTag'):
                    print "Jet TauTag Not implementet yet"
                if VaOfIn.has_key('Charge'):
                    print "Jet Charge Not implementet yet"
                if VaOfIn.has_key('EhadOverEem'):
                    print "Jet EhadOverEem Not implementet yet"
                if VaOfIn.has_key('NCharged'):
                    print "Jet NCharged Not implementet yet"
                if VaOfIn.has_key('NNeutrals'):
                    print "Jet NNeutrals Not implementet yet"
            #END IF JETS >0
        #END IF JET KEY

        
        # Cuts over Leptons and Photons
        # Made general
        #creo una lista de keys de leptones y corro sobre la lista
        """
        Entries	Number of Muons in the event
        PT	muon transverse momentum
        Eta	muon pseudorapidity
        Phi	muon azimuthal angle
        T	particle arrival time of flight
        Charge	muon charge
        Particle	reference to generated particle
        IsolationVar	isolation variable
        IsolationVarRhoCorr	isolation variable
        SumPtCharged	isolation variable
        SumPtNeutral	isolation variable
        SumPtChargedPU	isolation variable
        SumPt	isolation variable
        """
        if LsOfCuts.has_key('Muon'): 
            VaOfIn=LsOfCuts['Muon']                      #Which cuts over jets
            if VaOfIn.has_key('Entries'):
                Cuts=VaOfIn['Entries']
                for cut_ in Cuts:               #run over all the Entries cuts 
                    flag=False
                    filt=cut_
                    #Evaluate the condition
                    mycode="flag = True if PhyObj['Muon'].GetEntries() "+filt+" else False" 
                    exec(mycode)
                    name="Count_MuonEntries"+filt
                    if flag:                    #Counts
                        if counts.has_key(name):
                            counts[name] += 1
                        else:
                            counts[name] = 1
                    #CutFlow    
                        flags[name] = True  
                    else:
                        flags[name] = False        
                            
            if VaOfIn.has_key('PT') and PhyObj["Muon"].GetEntries() > 0:        #If ther are cut over PT
                Cuts=VaOfIn['PT']
                for cut_ in Cuts:               #run over all the PT cuts 
                    if isinstance(cut_, tuple):     #See if is tuple
                        flag=False
                        filt=cut_[1]
                        #See if there are enough entries to evaluate
                        if(PhyObj["Muon"].GetEntries() > cut_[0]):
                            muon= PhyObj["Muon"].At(cut_[0])
                            #Evaluate the condition
                            mycode="flag = True if muon.PT "+filt+" else False" 
                            exec(mycode)
                            name="Count_Muon["+str(cut_[0])+"]PT"+cut_[1]
                        if flag:                    #Counts
                            if counts.has_key(name):
                                counts[name] += 1
                            else:
                                counts[name] = 1
                        #CutFlow    
                            flags[name] = True  
                        else:
                            flags[name] = False     
                                
                    else:           #Evaluate the condition for the avaible jets 
                        for l in range(0, PhyObj["Muon"].GetEntries() ):
                            muon = PhyObj["Muon"].At(l)
                            flag=False
                            filt=cut_
                            #Evaluate the condition
                            mycode="flag = True if muon.PT "+filt+" else False" 
                            exec(mycode)
                            name="Count_Muon["+str(l)+"]PT"+filt
                            if flag:                    #Counts
                                if counts.has_key(name):
                                    counts[name] += 1
                                else:
                                    counts[name] = 1
                            #CutFlow    
                                flags[name] = True  
                            else:
                                flags[name] = False       
                                    
     
            if VaOfIn.has_key('Eta'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('Phi'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('T'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('Charge'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('Particle'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('IsolationVar'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('IsolationVarRhoCorr'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('SumPtCharged'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('SumPtNeutral'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('SumPtChargedPU'):
                print "Muon  Not implementet yet"
            if VaOfIn.has_key('SumPt'):
                print "Muon  Not implementet yet"
         
        
        # Cuts over MET
        """ 
        MET	mising transverse energy
        Eta	mising energy pseudorapidity
        Phi	mising energy azimuthal angle
        """
        if LsOfCuts.has_key('MissingET'):                   #Cuts over MissingET?
            VaOfIn=LsOfCuts['MissingET']                      #Which cuts over jets
            if VaOfIn.has_key('MET') and PhyObj["MissingET"].GetEntries() > 0:        #If ther are cut over PT
                Cuts=VaOfIn['MET']
                for cut_ in Cuts:               #run over all the PT cuts 
                    flag=False
                    filt=cut_
                    met= PhyObj["MissingET"].At(0)
                    #Evaluate the condition
                    mycode="flag = True if met.MET "+filt+" else False" 
                    exec(mycode)
                    name="Count_MissingETMET"+filt
                    if flag:                    #Counts
                        if counts.has_key(name):
                            counts[name] += 1
                        else:
                            counts[name] = 1     
                        #CutFlow    
                        flags[name] = True  
                    else:
                        flags[name] = False      

            if VaOfIn.has_key('Eta'):
                print "MissingET Eta Not implementet yet"
            if VaOfIn.has_key('Phi'):
                print "MissingET Phi Not implementet yet"

       

        #Calue vector, see how many of the conditions are fulfilled
        value=[]
        flagFlow={}
        if Debug:
            print "\n FlagFLOW..."
        for i in CutFlowUser:
            if Debug:
                print "CutFlowUser"+i
            if flags.has_key(i):
                if Debug:
                    print "Flag: "+str(flags[i])
                if flags[i]:
                    value.append(True)
                    flagFlow[i]=True
                else:
                    value.append(False)
                    flagFlow[i]=False
            else:
                value.append(False)
                flagFlow[i]=False
        
        #Sumary of all
        CutFlow[entry]=flagFlow
        
        #Fill the histograms
        if Debug:
            print "Filling the Histograms..."
        if sum(value)==Validationlen:
            if Debug:
                print "Pass"
            for m in CutUserFlagsObj:
                if Debug:
                    print "\nObjet: "+m
                for l in range(len(LsOfPOandVOI[m])):
                    if Debug:
                        print "Variable Of Interes: "+str(LsOfPOandVOI[m][l])
                    tmpval=0
                    tmpval1=0
                    tmpval2=0
                    if isinstance(LsOfPOandVOI[m][l], tuple):
                        if Debug:
                            print "Tuple"
                        name=m+"["+str(LsOfPOandVOI[m][l][0])+"]"+LsOfPOandVOI[m][l][1]
                        tmpval1="tmpval = PhyObj['"+m+"'].At("+str(LsOfPOandVOI[m][l][0])+")"
                        exec(tmpval1)
                        mycode="tmpval2 = tmpval."+LsOfPOandVOI[m][l][1]
                        if Debug:
                            print "name: "+name
                            print "Command: "+mycode
                        try:   
                            exec(mycode)
                        except AttributeError:
                            tmpval2=0
                        if Histograms.has_key(name)==False:
                            Histograms[name]=ROOT.TH1F()
                            Histograms[name].SetName(name)
                            Histograms[name].SetTitle(name)
                        Histograms[name].Fill(tmpval2)
                        
                    else:
                        if m=="Jet" or m=="Muon" or m=="Electron" or m=="Photon":
                            if LsOfPOandVOI[m][l] != "Entries":
                                name=m+"[0]"+LsOfPOandVOI[m][l]
                                tmpval1="tmpval = PhyObj['"+m+"'].At(0)"
                                exec(tmpval1)
                                mycode="tmpval2 = tmpval."+LsOfPOandVOI[m][l]
                                if Debug:
                                    print "name: "+name
                                    print "Command: "+mycode
                                try:   
                                    exec(mycode)
                                except AttributeError:
                                    tmpval2=0
                                if Histograms.has_key(name)==False:
                                    Histograms[name]=ROOT.TH1F()
                                    Histograms[name].SetName(name)
                                    Histograms[name].SetTitle(name)
                                Histograms[name].Fill(tmpval2)    
                            else:
                                name=m+LsOfPOandVOI[m][l]
                                mycode="tmpval2 = PhyObj['"+m+"'].Get"+LsOfPOandVOI[m][l]+"()"
                                if Debug:
                                    print "name: "+name
                                    print "Command: "+mycode
                                try:   
                                    exec(mycode)
                                except AttributeError:
                                    tmpval2=0
                                if Histograms.has_key(name)==False:
                                    Histograms[name]=ROOT.TH1F()
                                    Histograms[name].SetName(name)
                                    Histograms[name].SetTitle(name)
                                Histograms[name].Fill(tmpval2)
                        else:
                            if m=="MissingET":
                                name=m+LsOfPOandVOI[m][l]
                                tmpval1="tmpval = PhyObj['"+m+"']"
                                exec(tmpval1)
                                mycode="tmpval2 = tmpval.At(0)."+LsOfPOandVOI[m][l]
                                if Debug:
                                    print "name: "+name
                                    print "Command: "+mycode
                                try:   
                                    exec(mycode)
                                except AttributeError:
                                    tmpval2=0
                                if Histograms.has_key(name)==False:
                                    Histograms[name]=ROOT.TH1F()
                                    Histograms[name].SetName(name)
                                    Histograms[name].SetTitle(name)
                                Histograms[name].Fill(tmpval2)
                            else:
                                name=m+LsOfPOandVOI[m][l]
                                tmpval1="tmpval = PhyObj['"+m+"']"
                                exec(tmpval1)
                                mycode="tmpval2 = tmpval."+LsOfPOandVOI[m][l]
                                if Debug:
                                    print "name: "+name
                                    print "Command: "+mycode
                                try:   
                                    exec(mycode)
                                except AttributeError:
                                    tmpval2=0
                                if Histograms.has_key(name)==False:
                                    Histograms[name]=ROOT.TH1F()
                                    Histograms[name].SetName(name)
                                    Histograms[name].SetTitle(name)
                                Histograms[name].Fill(tmpval2)
                    if Debug:
                        print "temp= "+str(tmpval2)
        
     
    #Save the info 
    #Save the cuts counts        
    save_obj(counts, "CutsCounts" )
    save_obj(CutFlow, "CutFlow" )

** WARNING: cannot access branch 'Muon', return NULL pointer
** WARNING: cannot access branch 'MissingET', return NULL pointer
** WARNING: cannot access branch 'Jet', return NULL pointer


In [21]:
a=Histograms['JetEntries']

KeyError: 'JetEntries'

In [18]:
a.Draw()

NameError: name 'a' is not defined

In [ ]:
CutFlow

In [ ]:
CutFlowUser

In [ ]:
Validationlen

In [ ]:
value=[True, True, True, True]

In [ ]:
Debug

In [ ]:
tmpval

In [ ]:
CutUserFlagsObj

In [ ]:
Histograms.has_key("camilo")==False

In [ ]:
histoKeys=Histograms.keys()
for j in histoKeys:
    #print Histograms[j]
    mycode="flag = True if jet.PT "+filt+" else False" 
    print mycode
    #exec(mycode)

In [ ]:
histoKeys[0]

In [ ]:
a=Histograms[j]

In [ ]:
a.Fill()

In [ ]:
CutUserFlagsObj= LsOfPOandVOI.keys()
LsOfPOandVOI

In [ ]:
LsOfPOandVOI['Muon']

In [ ]:
CutUserFlagsObj= LsOfPOandVOI.keys()
for m in CutUserFlagsObj:
    #print m
    #print LsOfPOandVOI[m]
    for l in range(len(LsOfPOandVOI[m])):
        #print
        
        if isinstance(LsOfPOandVOI[m][l], tuple):
            tupla=LsOfPOandVOI[m][l]
            name="Count_"+m+"["+str(LsOfPOandVOI[m][l][0])+"]"+LsOfPOandVOI[m][l][1]
            print name
        else:
            name="Count_"+m+LsOfPOandVOI[m][l]
            print name


In [ ]:
LsOfPOandVOI['Muon'][0][0]

In [ ]:
LsOfPOandVOI[m][l]

In [ ]:
len(LsOfPOandVOI[m])

In [ ]:
CutUserFlagsObj=0

In [ ]:
CutUserFlagsObj

In [ ]:
CutFlags=flags.keys()

In [ ]:
CutFlags

In [ ]:
CutFlowUser=[]
PhOb=LsOfCuts.keys()
#print "PhOb", PhOb
#If is empty return error
for i in PhOb:
    VaOfIn=LsOfCuts[i]
    #If is empty return error
    #print VaOfIn
    Cuts=VaOfIn.keys()
    #If is empty return error
    #print "Cuts", Cuts
    for j in Cuts:
        cut_base=VaOfIn[j]
        for cut_ in cut_base:
            if isinstance(cut_, tuple):
                #print i+j+cut_[1]
                CutFlowUser.append("Count_"+i+"["+str(cut_[0])+"]"+j+cut_[1])
            else:
                #print i+j+cut_
                if i=="Jet" or i=="Muon" or i=="Electron" or i=="Photon":
                    if j != "Entries":
                        CutFlowUser.append("Count_"+i+"[0]"+j+cut_)
                    else:
                        CutFlowUser.append("Count_"+i+j+cut_)
                else:
                    CutFlowUser.append("Count_"+i+j+cut_)


In [ ]:
Validationlen=len(CutFlowUser)

In [ ]:
flags

In [ ]:
flags['Count_Muon[0]PT>5']

In [ ]:
flags.keys()
CutFlow={}
value=[]
for i in CutFlowUser:
    print i
    if flags.has_key(i):
        print flags[i]
        if flags[i]:
            value.append(True)
        else:
            value.append(False)
    else:
        value.append(False)

In [ ]:
sum(value)

In [ ]:
        #Create the plots
        Histograms={}
        CutUserFlagsObj= LsOfPOandVOI.keys()
        for m in CutUserFlagsObj:
        #print m
        #print LsOfPOandVOI[m]
            for l in range(len(LsOfPOandVOI[m])):
            #print
                if isinstance(LsOfPOandVOI[m][l], tuple):
                    name=m+"["+str(LsOfPOandVOI[m][l][0])+"]"+LsOfPOandVOI[m][l][1]
                    #print name
                    Histograms[name]=ROOT.TH1F()
                else:
                    if m=="Jet" or m=="Muon" or m=="Electron" or m=="Photon":
                        if LsOfPOandVOI[m][l] != "Entries":
                            name=m+"[0]"+LsOfPOandVOI[m][l]
                            #print name
                            Histograms[name]=ROOT.TH1F()
                            Histograms[name].SetName(name)
                            Histograms[name].SetTitle(name)
                        else:
                            name=m+LsOfPOandVOI[m][l]
                            #print name
                            Histograms[name]=ROOT.TH1F()
                    else:
                        name=m+LsOfPOandVOI[m][l]
                        #print name
                        Histograms[name]=ROOT.TH1F()

In [ ]:
Histograms

In [ ]:
a=Histograms['JetEntries']

In [ ]:
a.SetName('JetEntries')
a.Fill

In [ ]:
# Book histograms
histJetPT = ROOT.TH1F("jet_pt", "jet P_{T}", 100, 0.0, 100.0)
histMass = ROOT.TH1F("mass", "M_{inv}(e_{1}, e_{2})", 100, 40.0, 140.0)
histJetPT.Fill(jet.PT)

